In [1]:
)clear
⎕IO←0      ⍝ index origin
⎕CT←1e¯12  ⍝ comparison tolerance
⎕PP←4      ⍝ print precision

clear ws

# Quaternions in Dyalog APL

## 3D orientations

There are different ways in which to represent the orientation of a three-dimensional object with respect to a reference.

### Euler angles

Three Euler angles represent the orientation of the object with respect to the reference system as three successive rotations. Different conventions can be used.

In [classic Euler angles](https://en.wikipedia.org/wiki/Euler_angles#Classic_Euler_angles), three successive rotations $\left(\varphi_1, \Phi, \varphi_2\right)$ are performed along the axis $ZXZ$. The first angle $\varphi_1$ is the rotation along the original $Z$ axis, the second angle $\Phi$ is the rotation along the axis $X'$ (the result of rotating $X$), the third angle $\varphi_2$ is the rotation along $Z'$ (the $Z$ axis after having been rotated by $\Phi$). This convention is commonly used in crystallography.

[Tait-Bryan angles](https://en.wikipedia.org/wiki/Euler_angles#Tait%E2%80%93Bryan_angles) represent three successive rotations with angles $\left(\alpha, \beta, \gamma\right)$ along the axes $XYZ$. This other convention is used in aerospacial applications.

It is relatively simple to find what is the orientation of a 3D body when given the Euler angles. However, Euler angles are difficult to compose, so other representations like matrices or quaternions are employed in software.

### Quaternions

A unitary quaternion allows to reperesent a 3D rotation using four values. The first one is a factor that multiplies the original vector, and the other three indicate the component of the rotated vector perpendicular to the original one. In practice, there would be an ambiguity for rotations of 180 degrees (the same quaternion would correspond to a rotation of 180 degrees along any axis). Therefore, when used to represent orientations, quaternions of half the angle are used.

At difference of Euler angles, quaternions present a number of interesting algebraic properties that make manipulating them and working with them very convenient, so they are commonly used in 3D software.

## 3D orientations in APL

### Vectors

Vectors will be represented using one-dimensional arrays.

In [2]:
⍝ base unitary 3D vectors
⊢(x y z)←=∘⊂⍨⍳3

┌─────┬─────┬─────┐
│1 0 0│0 1 0│0 0 1│
└─────┴─────┴─────┘

The elements of the array might be anything (including other arrays) as long as the elements conform.

In [3]:
⍝ eg: vectors
⍕¨a←?4⍴0         ⍝ 4D vector with scalar components
⍕¨b←?4⍴⊂3⍴0      ⍝ 4D vector with list components
⍕¨c←?4⍴⊂3 2⍴0    ⍝ 4D vector with matrix components
⍕¨d←0(?0)@0 2⊢c  ⍝ 4D vector mixing matrix and scalar components

┌──────┬──────┬──────┬──────┐
│0.4435│0.6429│0.9009│0.5479│
└──────┴──────┴──────┴──────┘

┌────────────────────┬────────────────────┬────────────────────┬───────────────────────┐
│0.4494 0.6435 0.1089│0.4493 0.9956 0.3286│0.3417 0.8475 0.7488│0.04295 0.5268 0.004596│
└────────────────────┴────────────────────┴────────────────────┴───────────────────────┘

┌───────────────┬─────────────┬─────────────┬───────────────┐
│0.17 0.757 │0.032 0.4944│0.3587 0.9482│0.005923 0.492 │
│0.843 0.1468 │0.3831 0.9079│0.6896 0.3902│0.4128 0.2964│
│0.6448 0.002345│0.2117 0.18 │0.6242 0.3592│0.5839 0.5634│
└───────────────┴─────────────┴─────────────┴───────────────┘

┌─┬─────────────┬──────┬───────────────┐
│0│0.032 0.4944│0.5849│0.005923 0.492 │
│ │0.3831 0.9079│ │0.4128 0.2964│
│ │0.2117 0.18 │ │0.5839 0.5634│
└─┴─────────────┴──────┴───────────────┘

We will need some small helper functions to work with vectors. The function `U` makes a vector unitary dividing it by its magnitude, calculated with `M`.

In [4]:
M ← (÷2)*⍨+.×⍨  ⍝ Magnitude of vector
U ← ×∘(÷M)⍨     ⍝ Unitary vector

We can use these functions to calculate unitary vectors from our examples and check that their magnitude is indeed 1.

In [5]:
⍕¨↑U¨a b c d     ⍝ unitary vectors
(1=M)∘U¨a b c d  ⍝ check that their magnitude is 1

┌───────────────────┬────────────────────┬────────────────────┬─────────────────────┐
│0.338 │0.49 │0.6866 │0.4175 │
├───────────────────┼────────────────────┼────────────────────┼─────────────────────┤
│0.6218 0.4153 0.132│0.6216 0.6425 0.3983│0.4727 0.5469 0.9077│0.05943 0.34 0.005571│
├───────────────────┼────────────────────┼────────────────────┼─────────────────────┤
│0.4269 0.5409 │0.08034 0.3533 │0.9006 0.6775 │0.01487 0.3516 │
│0.6875 0.1409 │0.3125 0.8712 │0.5624 0.3745 │0.3367 0.2844 │
│0.5908 0.00339 │0.1939 0.2602 │0.5719 0.5191 │0.535 0.8142 │
├───────────────────┼────────────────────┼────────────────────┼─────────────────────┤
│0 0 │0.05462 0.5431 │0.9985 0.6425 │0.01011 0.5405 │
│0 0 │0.4718 0.8107 │0.7203 0.5222 │0.5085 0.2647 │
│0 0 │0.2481 0.2164 │0.6855 0.7031 │0.6845 0.6773 │
└───────────────────┴────────────────────┴────────────────────┴─────────────────────┘

┌─┬───────┬─────┬─────┐
│1│┌─────┐│┌───┐│┌───┐│
│ ││1 1 1│││1 1│││1 1││
│ │└─────┘││1 1│││1 1││
│ │ ││1 1│││1 1││
│ │ │└───┘│└───┘│
└─┴───────┴─────┴─────┘

### Euler angles in APL

A triplet of Euler angles can be easily represented in APL using an array of three elements. Like with vectors, we can represent multiple Euler angles using nested arrays as components:

In [6]:
⍕¨ea←30 60 45                ⍝ triplet of Euler angles
⍕¨ea1←↓180+-360×?3 3⍴0       ⍝ 3 triplets of Euler angles
⍕¨ea2←⊂⍤2⊢180+-360×?3 4 2⍴0  ⍝ 4×2 triplets of Euler angles

┌──┬──┬──┐
│30│60│45│
└──┴──┴──┘

┌─────────────────┬───────────────────┬───────────────────┐
│177.1 81.98 159.4│¯148.9 168.6 0.1939│¯163.8 ¯111.7 14.35│
└─────────────────┴───────────────────┴───────────────────┘

┌──────────────┬──────────────┬──────────────┐
│ ¯25.04 121.2 │¯117.1 149.1 │ 32.71 33.15│
│¯121.8 ¯58.14│ 133.9 ¯64.83│¯127.9 ¯53 │
│ 12.17 39.72│ 148.7 115.8 │¯104.9 12.35│
│ 66.78 88.58│ 70.95 ¯43.84│ ¯26.22 81.79│
└──────────────┴──────────────┴──────────────┘

### Quaternions in APL

Quaternions will be represented as arrays of four elements. As with vectors and Euler angles, we will use array components to represent multiple values (actually, the quaternions space $\mathbb{H}$ is a vector space, so quaternions are 4D vectors).

In [7]:
⍝ sample unitary quaternions
⍕¨q←U?4⍴0
⍕¨q1←U↓?4 3⍴0
⍕¨q2←U⊂⍤2?4 3 2⍴0

┌──────┬──────┬──────┬──────┐
│0.2734│0.7335│0.1734│0.5977│
└──────┴──────┴──────┴──────┘

┌────────────────────┬────────────────────┬─────────────────────┬────────────────────┐
│0.07985 0.6807 0.659│0.1491 0.4287 0.7441│0.0466 0.4962 0.03197│0.9845 0.3266 0.1052│
└────────────────────┴────────────────────┴─────────────────────┴────────────────────┘

┌─────────────┬───────────────┬─────────────┬──────────────┐
│0.1029 0.3949│0.04269 0.09856│0.5552 0.8732│0.8242 0.2681 │
│0.0731 0.6392│0.0677 0.3116 │0.8412 0.699 │0.5315 0.07564│
│0.2993 0.7118│0.1226 0.2311 │0.7598 0.2091│0.564 0.6295 │
└─────────────┴───────────────┴─────────────┴──────────────┘

#### Random orientations

Although the quaternions defined above are random, if we generate random quaternions in this way they will not be uniformly distributed. We want a function that can generate a number of quaternions that actually represent random orientations in the 3D space.

In order to generate a random orientation, what we want is to choose a random point from the four-dimensional sphere that contains the unitary quaternions. We first select a random latitude, then we rotate two randomly chosen angles. The function `RND` will do this to generate quaternions with the given shape and magnitude (with default magnitude 1).

In [8]:
RND←{⍺←1 ⋄ F←{(1 2○2×⊂○?⍺⍴0)×⊂⍵*÷2} ⋄ ⍺×U⍵(F∘(1∘-),F)?⍵⍴0}
⍕¨RND 3    ⍝ list
⍕¨RND 3 2  ⍝ array
⍕¨RND ⍬    ⍝ scalar (empty shape)

┌──────────────────────┬───────────────────────┬─────────────────────┬───────────────────┐
│¯0.01367 0.9193 0.2717│¯0.2583 ¯0.1792 ¯0.4448│0.5498 0.1308 ¯0.8431│0.7942 ¯0.325 0.132│
└──────────────────────┴───────────────────────┴─────────────────────┴───────────────────┘

┌────────────────┬────────────────┬────────────────┬────────────────┐
│ 0.9798 0.06732│ 0.124 ¯0.4876 │ 0.04492 ¯0.8702│ 0.1504 ¯0.02141│
│¯0.3759 ¯0.4255 │¯0.2307 0.1525 │¯0.5506 0.8851│ 0.7087 ¯0.1106 │
│¯0.7143 ¯0.4677 │ 0.5786 ¯0.08823│¯0.2062 ¯0.2577│¯0.3354 0.8409 │
└────────────────┴────────────────┴────────────────┴────────────────┘

┌───────┬──────┬───────┬───────┐
│¯0.6259│0.5637│¯0.2986│¯0.4487│
└───────┴──────┴───────┴───────┘

## Transformation from Euler angles into quaternions

Given a triplet of Euler angles, we can calculate the corresponding quaternion for each of the three rotations and then compose the rotations multipling the quaternions. We will need a function to get a quaternion from a rotation represented as an axis and an angle, and a function to multiply quaternions.

### Quaternion from axis-angle

We start with the function `A` to get a quaternion from an axis and an angle. The scalar component of the quaternion is the cosine of the half angle, and its vector component is a vector with magnitude the sine of the half angle and the direction of the rotation axis. The axis will be the left argument (default `z`) and the angle, in radians, will be the right argument. The function `AD` is analogous to `A` but it takes angles in degrees.

In [9]:
A←{⍺←z ⋄ (⊂2○⍵),(⊂1○⍵)∘.×U⍺}∘(÷∘2) ⋄ AD←A∘(180÷⍨○)  ⍝ quaternion from axis-angle in radians and degrees

For example:

In [10]:
⍪¨  A⊃ea1  ⍝ rotations around z
⍪¨x A⊃ea1  ⍝ rotations around x

┌─────────────┬─┬─┬─────────────┐
│ 0.8258346996│0│0│ 0.5639122705│
│¯0.9885406108│0│0│¯0.1509551615│
│¯0.4042091512│0│0│¯0.9146665852│
└─────────────┴─┴─┴─────────────┘

┌─────────────┬─────────────┬─┬─┐
│ 0.8258346996│ 0.5639122705│0│0│
│¯0.9885406108│¯0.1509551615│0│0│
│¯0.4042091512│¯0.9146665852│0│0│
└─────────────┴─────────────┴─┴─┘

### Product of quaternions

Next, we need a quaternion product.

The multiplication table for quaternion components is defined (as shown in certain [bridge](https://en.wikipedia.org/wiki/Broom_Bridge)) as:

In [11]:
' -'[↑(1 1 1 1)(1 ¯1 1 ¯1)(1 ¯1 ¯1 1)(1 1 ¯1 ¯1)<0],¨'1ijk'[↑(0 1 2 3)(1 0 3 2)(2 3 0 1)(3 2 1 0)]

┌──┬──┬──┬──┐
│ 1│ i│ j│ k│
├──┼──┼──┼──┤
│ i│-1│ k│-j│
├──┼──┼──┼──┤
│ j│-k│-1│ i│
├──┼──┼──┼──┤
│ k│ j│-i│-1│
└──┴──┴──┴──┘

Our multiplication function will use the [Hamilton product](https://en.wikipedia.org/wiki/Quaternion#Hamilton_product). First, we will index the components of the right quaternion that multiply the sorted components of the left quaternion (stored in the variable `pi`) and multiply them. Then, we multiply by the signs (in the variable `pu`) and sum over the first axis.

In [12]:
pi←↑(0 1 2 3)(1 0 3 2)(2 3 0 1)(3 2 1 0)        ⍝ product indices
pu←↑(1 ¯1 ¯1 ¯1)(1 1 1 ¯1)(1 ¯1 1 1)(1 1 ¯1 1)  ⍝ product unit factors
P←+/pu×⊣(×⍤1)(⊂pi)⌷⊢                            ⍝ product function

Let's perform some checks:

In [13]:
(u i j k)←=∘⊂⍨⍳4                  ⍝ base units
((⊂-u)≡¨P¨⍨i j k)∧(-u)≡i P j P k  ⍝ check that what Hamilton wrote on Broom bridge holds

1 1 1

### Quaternions from Euler angles

Now, we can define the function `E` to transform Euler angles into quaternions. The function will need to calculate the quaternions corresponding to the three rotations and multiply them. By default, we will consider that the three rotations correspond to the classic Euler angles (`z x z`). Additionally, we define an `ED` function that will take angles in degrees.

In [14]:
E←{⍺←z x z ⋄ ⊃⍺P.A⍥⌽⍵} ⋄ ED←E∘(180÷⍨○)  ⍝ quaternion from Euler angles (default zxz) in radians and degrees

Now, we can transform into quaternions the Euler angles that we defined before:

In [15]:
⍕¨ED ea
⍕¨ED ea1
⍕¨ED ea2

┌──────┬──────┬───────┬──────┐
│0.6871│0.4957│0.06526│0.5272│
└──────┴──────┴───────┴──────┘

┌──────────────────────┬────────────────────────┬───────────────────────┬───────────────────────┐
│0.2659 0.09587 0.05459│0.9501 ¯0.1183 0.0005084│0.1599 ¯0.988 ¯0.001614│0.03114 ¯0.02541 0.9985│
└──────────────────────┴────────────────────────┴───────────────────────┴───────────────────────┘

┌───────────────┬───────────────┬─────────────────┬────────────────┐
│ 0.5207 0.05904│¯0.747 0.6931│¯0.4119 ¯0.67 │ 0.03492 0.2594│
│¯0.2238 0.4773 │ 0.9189 ¯0.5355│¯0.04889 ¯0.02402│¯0.3213 ¯0.6963│
│ 0.1861 0.4773 │ 0.5029 0.8232│¯0.8212 ¯0.2004 │¯0.1951 0.2332│
│ 0.7639 0.0779 │ 0.3995 ¯0.3727│¯0.421 0.02211│ 0.2823 0.9244│
└───────────────┴───────────────┴─────────────────┴────────────────┘

## Additional operations

### Quaternion product operator

As we have seen, when we apply our product function to quaternions with non-scalar components, conformability and scalar extension are applied. However, there will be situations in which we will want to perform some other operation different from the scalar product.

Let's define an operator analogous to the `P` function which can take any product function as left operand:

In [16]:
_P←{(i u)←⍵⍵ ⋄ +/⍺(⍺⍺¨⍤1)u×(⊂i)⌷⍵}pi pu  ⍝ product operator

With this operator, we can perform dot and outer products, for example.

In [17]:
⍕¨b+.×_P c  ⍝ dot product of vector and matrix
⍕¨b∘.×_P c  ⍝ outer product of vector and matrix
a(P≡×_P)b   ⍝ check that the product P is equivalent to ×_P

┌─────────────┬───────────┬────────────┬────────────┐
│¯1.171 ¯1.854│1.819 1.906│1.527 0.9003│1.345 0.4425│
└─────────────┴───────────┴────────────┴────────────┘

┌────────────────┬───────────────┬───────────────┬──────────────────┐
│¯0.06077 ¯0.2271│0.07739 0.6897│0.218 0.485 │ 0.1602 0.5108 │
│¯0.04662 ¯0.488 │0.6624 0.5585│0.4289 0.1314 │ 0.4007 0.00464 │
│¯0.04364 ¯0.2268│0.5576 0.259 │0.2476 ¯0.08317│ 0.4983 0.3532 │
│ │ │ │ │
│¯0.2295 ¯1.068 │0.005928 0.9892│0.3859 1.022 │ 0.4234 1.24 │
│¯0.6409 ¯1.296 │1.072 0.7761│0.949 0.5588 │ 1.072 ¯0.1129 │
│¯0.6324 ¯0.7789│0.9443 0.4064│0.4784 ¯0.2329 │ 1.158 0.5688 │
│ │ │ │ │
│¯0.2606 ¯0.7923│0.06214 0.6666│0.1646 0.5107 │ 0.09532 ¯0.001584│
│¯0.5523 ¯0.5759│0.6247 0.3673│0.5724 0.05921│¯0.01147 ¯0.5187 │
│¯0.4694 ¯0.3304│0.6693 0.4406│0.3599 ¯0.1434 │ 0.1131 0.04457 │
└────────────────┴───────────────┴───────────────┴──────────────────┘

1

### Conjuagate quaternion and scalar difference

In addition to a quaternion product, it is interesting to dispose of a conjugate function (`C`) which can return the conjugate of a quaternion. The conjugate of a unitary quaternion represents the inverse rotation, and it is obtained negating the non-scalar components (which is equivalent to inverting the axis).

The scalar difference is defined as the scalar part of the quaternion representing the relative rotation between two given quaternions. It will be used later for the calculation of misorientation angles.

In [18]:
C←×∘(1,-3⍴1)  ⍝ conjugate quaternion
D←+.×∘C       ⍝ scalar difference

### Rotations and misorientations

First, we will define an operator so that we can easily work either with quaternions or with vectors of up to 3 dimensions. The operator will check if any of the arguments has a length lower than 4 and, in that case, it will consider that it is a vector, if the argument is a list, or a scalar, if it is a scalar value. Vectors are represented using the imaginary components of the quaternion (the last three components).

In [19]:
]dinput
_←{⍺←⊢
    0::(⊃⍬⍴⎕DM)⎕SIGNAL ⎕EN                                   ⍝     throw all errors
    ⍬≡⍴⍵:⍺∇4↑⍤1⊢⍵ ⋄ 4>⊃⌽⍴⍵:⍺∇(¯4↑3↑⊢)⍤1⊢⍵ ⋄ 3=⎕NC'⍺':⍺⍺⍤1⊢⍵  ⍝     take vectors as right argument
    ⍬≡⍴⍺:(4↑⍤1⊢⍺)∇⍵ ⋄ 4>⊃⌽⍴⍺:((¯4↑3↑⊢)⍤1⊢⍺)∇⍵ ⋄ ⍺ ⍺⍺⍤1⊢⍵     ⍝     take vectors as left argument
}

Let's try it:

In [20]:
⊢_¨0.1(,1)(1 2)(1 2 3)  ⍝ scalar, 1D vector, 2D vector and 3D vector
0.1 P _ 1 2 3           ⍝ product by scalar is scalar product

┌─────────┬───────┬───────┬───────┐
│0.1 0 0 0│0 1 0 0│0 1 2 0│0 1 2 3│
└─────────┴───────┴───────┴───────┘

0 0.1 0.2 0.3

Now, we use this operator to define functions capable of rotating quaternions and vectors.

In [21]:
R←(⊣P P∘C⍨)_                                       ⍝ rotate
R2←(⊂1+⍳2)⌷⍤1{A⍺}R⊢ ⋄ R2D←(⊂1+⍳2)⌷⍤1{AD⍺}R⊢        ⍝ rotation of 2D vectors
R3←(⊂1+⍳3)⌷⍤1{⊃A⍨/⍺}R⊢ ⋄ R3D←(⊂1+⍳3)⌷⍤1{⊃AD⍨/⍺}R⊢  ⍝ rotation of 3D vectors    

For example:

In [22]:
(AD 90)R,1          ⍝ rotate 90 degrees around z the vector 1 0 0 (return quaternion)
90 R2D 1 0          ⍝ rotate 90 degrees around z the vector 1 0 0 (return 2D vector)
90(0 0 1)R3D 1 0 0  ⍝ rotate 90 degrees around z the vector 1 0 0 (return 3D vector)

0 0 1 0

0 1

0 1 0

We also define the functions `∆` and `∆D`, which we will use to calculate the angle between two vectors or quaternions.

In [23]:
∆←(¯2○D∘C⍥U)_ ⋄ ∆D←(180÷○1)×∆

So:

In [24]:
0 1 0 ∆D 1 0 0

90

## Summary

We put together everything we have done in the namespace `H` (the H is for Hamilton).

In [25]:
:Namespace H                                                         ⍝ Quaternions. jgl@dyalog.com 2023-2024
    ⎕IO←0
    i←↑(0 1 2 3)(1 0 3 2)(2 3 0 1)(3 2 1 0)                          ⍝ product indices
    u←↑(1 ¯1 ¯1 ¯1)(1 1 1 ¯1)(1 ¯1 1 1)(1 1 ¯1 1)                    ⍝ product unit factors
    _P←{(i u)←⍵⍵ ⋄ +/⍺(⍺⍺¨⍤1)u×(⊂i)⌷⍵}i u ⋄ P←+/u×⊣(×⍤1)(⊂i)⌷⊢       ⍝ hamilton product operator and function
    C←×∘(1,-3⍴1) ⋄ D←+.×∘C ⋄ M←(÷2)*⍨+.×⍨ ⋄ U←×∘(÷M)⍨                ⍝ conjugate, difference, magnitude, unitary
    A←{⍺←z ⋄ (⊂2○⍵),(⊂1○⍵)∘.×U⍺}∘(÷∘2) ⋄ AD←A∘(180÷⍨○)               ⍝ quaternion from axis-angle (z)
    E←{⍺←z x z ⋄ ⊃⍺P.A⍥⌽⍵} ⋄ ED←E∘(180÷⍨○)                           ⍝ quaternion from Euler angles (zxz)
    _←{⍺←⊢                                                           ⍝ apply to quaternion or vector components
        0::(⊃⍬⍴⎕DM)⎕SIGNAL ⎕EN                                       ⍝     throw all errors
        ⍬≡⍴⍵:⍺∇4↑⍤1⊢⍵ ⋄ 4>⊃⌽⍴⍵:⍺∇(¯4↑3↑⊢)⍤1⊢⍵ ⋄ 3=⎕NC'⍺':⍺⍺⍤1⊢⍵      ⍝     take vectors as right argument
        ⍬≡⍴⍺:(4↑⍤1⊢⍺)∇⍵ ⋄ 4>⊃⌽⍴⍺:((¯4↑3↑⊢)⍤1⊢⍺)∇⍵ ⋄ ⍺ ⍺⍺⍤1⊢⍵         ⍝     take vectors as left argument
    }
    R←{⍺(⊣P P∘C⍨)_⍵} ⋄ ∆←(¯2○D∘C⍥U)_ ⋄ ∆D←(180÷○1)×∆                 ⍝ rotate, relative angle, angle in degrees
    R2←(⊂1+⍳2)⌷⍤1{A⍺}R⊢ ⋄ R2D←(⊂1+⍳2)⌷⍤1{AD⍺}R⊢                      ⍝ rotation of 2D vectors
    R3←(⊂1+⍳3)⌷⍤1{⊃A⍨/⍺}R⊢ ⋄ R3D←(⊂1+⍳3)⌷⍤1{⊃AD⍨/⍺}R⊢                ⍝ rotation of 3D vectors    
    RND←{⍺←1 ⋄ F←{(1 2○2×⊂○?⍺⍴0)×⊂⍵*÷2} ⋄ ⍺×U⍵(F∘(1∘-),F)?⍵⍴0}       ⍝ random unitary quaternions    
    (u i j k)←=∘⊂⍨⍳4 ⋄ (x y z)←=∘⊂⍨⍳3                                ⍝ base units
 
    ∇ Test ;t;T;S;v;w;s;p;q;r;a;b;c;d;e;as
        ⎕CT←1E¯10 ⋄ t←0 ⋄ w←?0
        T←{0∊⍵:('TEST FAILED: ',⍺)⎕signal 8 ⋄ t+←1}
        'broom bridge'T((⊂-u)≡¨P¨⍨i j k)∧(-u)≡i P j P k
        'scalar difference'T(¯1≡¨D⍨¨i j k)∧(0≡¨i j k D¨j k i)
        :For s :In (⍬ 1 3 (2 3))
            (p q r)←RND¨3⍴⊆s ⋄ v←s∘⍴¨u ⋄ S←{⍵,' ',⍕s}
            (S'unit')T v≡⍥(1∘+)P∘C⍨q
            (S'neutral')T q∘≡¨(q P v)(v P q)
            (S'scalar')T(w×q P p)∘≡¨((w×q)P p)(q P w×p)
            (S'commutatative')T(q P w×v)≡(w×v)P q
            (S'associative')T(p P q P r)≡(p P q)P r
            (S'left distributive')T((p + q)P r)≡(p P r)+q P r
            (S'right distributive')T(p P q + r)≡(p P q)+p P r
            (S'random')T(((s⍴1)≡∘⊃+.×⍨)∧(⊂,s)∧.≡⍴¨)RND s
        :EndFor
        a←?4⍴0 ⋄ e←⊃b←?4⍴⊂3⍴0 ⋄ c←?4⍴⊂3 2⍴0 ⋄ d←0 w@0 2⊢c
        'conform scalar'T 1⊣(⊂a)P¨as←a b c d
        'conform list'T 1⊣(⊂b)P¨a b
        'conform matrix'T 1⊣(⊂c)P¨a c d
        'conform mixed'T 1⊣(⊂d)P¨a c d
        'operator'T((⊂a)P¨as)≡(⊂a)(×_P)¨as
        'inner product'T(3 3)(2 2)≡⍴∘⊃¨c(+.×_P,⍥⊂+.×_P⍨)⍉¨c
        'outer product'T 3 3 2≡⍴⊃b∘.×_P c
        'scalar extension'T(w×¨a b c)≡w P _¨a b c
        '1D-vector extension'T((⊂,1)P _¨as)≡(⊂0 1 0 0)P¨as
        '2D-vector extension'T((⊂1 2)P _¨as)≡(⊂0 1 2 0)P¨as
        '3D-vector extension'T((⊂1 2 3)P _¨as)≡(⊂0 1 2 3)P¨as
        'unitary'T(1∧.=∘∊M⍨)∘U¨a b c d
        'rotate'T(∘.R⍨i j k)≡3 3⍴i(-j)(-k)(-i)j(-k)(-i)(-j)k
        'rotate 2D'T(0 1)(0 ¯1)≡90 ¯90 R2D¨⊂1 0
        'rotate 3D'T(0 1 0)≡90(0 0 1)R3D 1 0 0
        'angle'T 0 90 180≡(1 0 0)(0 1 0)(¯1 0 0)∆D¨⊂1 0 0
        'euler'T(E e)≡⊃P/(0 0 1)(1 0 0)(0 0 1)A¨⌽e←⊃b
        ⎕←(⍕t),' TESTS PASSED'
    ∇
:EndNamespace

In [26]:
H.Test

51 TESTS PASSED